<a href="https://colab.research.google.com/github/Karanveer08/Pst_to_txt_converter/blob/main/PST_to_TXT_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install loguru

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.0 MB/s eta 0:00:00


In [ ]:
pip install libpff-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for libpff-python: filename=libpff_python-20211114-cp310-cp310-linux_x86_64.whl size=1837850 sha256=0fa902af764b5214ecd6a2d56e8971c1bf73b034076c5028f8ae8d6257ec97ab
  Stored in directory: /root/.cache/pip/wheels/72/7f/71/266369be569e85f399a25ad5ece87077b25f5e0d96d2582683
Successfully built libpff-python


In [ ]:
from loguru import logger

In [ ]:
import json
import os
import sys
import re

import pypff

In [ ]:
output_directory = ""
next_line = "\n"
hex_pattern = r"\\x[0-9a-fA-F*]{2}(?![@*])"
newline_pattern = r"\r\n"
exclusion_pattern = f"{hex_pattern}|{newline_pattern}"


root_directory = f"/content/drive/MyDrive/dataset"
response_output = ""

In [ ]:
# response_output = ""

date_dict = {x: 0 for x in range(1, 25)}
date_list = [date_dict.copy() for x in range(7)]

valid_extensions = [".pst"]

logger.add("./logs/file_{time}.log", level="TRACE", rotation="100 MB")

2

In [ ]:
def extract(output, pst_file):
    try:
        logger.debug("Opening PST for processing...")
        opst = pypff.open(pst_file)
        root = opst.get_root_folder()

        logger.debug("Starting traverse of PST structure...")
        folderTraverse(root, output)
    except Exception as ex:
        print(ex)

In [ ]:
def folderTraverse(base, output):
    try:
        if hasattr(base, "sub_folders"):
            for folder in base.sub_folders:
                if folder.number_of_sub_folders:
                    folderTraverse(folder, output)
                checkForMessages(folder, output)

    except Exception as ex:
        print(ex)

In [ ]:
def checkForMessages(folder, output):
    logger.debug("Processing Folder: " + folder.name)
    message_list = []
    for message in folder.sub_messages:
        processedMessage = processMessage(message)

        if processedMessage:
            message_dict = processedMessage
            message_list.append(message_dict)
    folderReport(message_list, output)


In [ ]:
def processMessage(message):
    cleaned_string = ""

    if message.plain_text_body:
        plain_text_body = message.plain_text_body.strip()

        if plain_text_body:
            cleaned_string = f"""{ plain_text_body } """

            # Decode the bytes to get a string
            # cleaned_string = str(cleaned_string.decode("utf-8")).strip()

    if len(cleaned_string) > 10:
        message = {
            "subject": str(message.subject),
            "sender": str(message.sender_name),
            "header": str(message.transport_headers),
            "body": cleaned_string,
            "creation_time": message.creation_time.strftime(
                "%Y-%m-%d %H:%M:%S"
            ),  # Format as a string
            "submit_time": message.client_submit_time.strftime(
                "%Y-%m-%d %H:%M:%S"
            ),  # Format as a string
            "delivery_time": message.delivery_time.strftime(
                "%Y-%m-%d %H:%M:%S"
            ),  # Format as a string
            "attachment_count": message.number_of_attachments,
        }

        json_message = json.dumps(message)
        return json_message
    else:
        return None

In [ ]:
def folderReport(message_list, file_path):
    if not len(message_list):
        logger.warning(f"Filename: {file_path} -->> Empty message not processed")
        return

    # extracted_files = f"{output_dir}{project_id}/"
    for index, message in enumerate(message_list):
        convert_json = json.loads(message)
        date_string = convert_json["creation_time"].replace(" ", "_").replace(":", "_")
        filename = f"{date_string}_{convert_json['sender']}"
        filename = re.sub(r"\W+", "_", filename)
        filename = f"{filename}_{index}.txt"
        file_output = os.path.join(file_path, filename)

        message = construct_message(file_output, message)
        """ # Creation Time
        day_of_week = message["creation_time"].weekday()
        hour_of_day = message["creation_time"].hour + 1
        date_list[day_of_week][hour_of_day] += 1
        # Submit Time
        day_of_week = message["submit_time"].weekday()
        hour_of_day = message["submit_time"].hour + 1
        date_list[day_of_week][hour_of_day] += 1
        # Delivery Time
        day_of_week = message["delivery_time"].weekday()
        hour_of_day = message["delivery_time"].hour + 1
        date_list[day_of_week][hour_of_day] += 1 """



In [ ]:
def write_to_local_data(file_name: str, file_content: str):
    try:
        if file_name and file_content:
            with open(file_name, "w") as fh:
                fh.write(file_content)
    except Exception as e:
        print(e)

In [ ]:
 def sanitize(message):
    split_value = str(message).split("\r\n")
    convert_to_string = "".join(split_value)

    return re.sub(exclusion_pattern, "", convert_to_string)


In [ ]:
def extract_emails(email_header):
    try:
        from_line = ""
        to_line = ""
        from_pattern = re.compile(r"From:\s*(.+)")
        to_pattern = re.compile(r"To:\s*(.+)")

        # Extract "From" and "To" lines
        from_match = from_pattern.search(email_header)
        to_match = to_pattern.search(email_header)

        if from_match:
            from_line = from_match.group(1)
            # print(f"From: {from_line}")

        if to_match:
            to_line = to_match.group(1)
            # print(f"To: {to_line}")

        return {"From": from_line, "To": to_line}
    except Exception as ex:
        return ""


In [ ]:
def construct_message(file_path, message):
    """_ = [
        f"sender: {sanitize(message['sender'])}{next_line}",
        f"subject: {sanitize(message['subject'])}{next_line}",
        f"header: {extract_emails(message['header'])}{next_line}",
        f"creation_time: {sanitize(message['creation_time'])}{next_line}",
        f"submit_time: {sanitize(message['submit_time'])}{next_line}",
        f"delivery_time: {sanitize(message['delivery_time'])}{next_line}",
        f"body: {sanitize(message['body'])}{next_line}",
        f"attachment_count: {sanitize(message['attachment_count'])}",
    ]

    output_message = next_line.join(_)"""

    try:
        write_to_local_data(file_path, message)
    except Exception as e:
        print(e)

    return message

In [ ]:
if __name__ == "__main__":
    project_id = 108

    print(len(sys.argv))
    if len(sys.argv) == 5:
        project_id = int(sys.argv[1])
    else:
        print("Insufficient Parameters!")

    print("Matter Id:", project_id)

    source_dir = f"{root_directory}"
    log_dir = f"{source_dir}/{project_id}"
    output_dir = f"{root_directory}"

    output_directory = os.path.realpath(output_dir)
    source_files = os.path.realpath(source_dir)
    log_director = os.path.abspath(log_dir)

    print("Source Dir: ", source_files)
    print("Output Dir: ", output_directory)

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    log_path = os.path.join(log_director, "pst_indexer.log")
    """ logger.basicConfig(
        filename=log_path,
        level=logger.DEBUG,
        format="%(asctime)s | %(levelname)s | %(message)s",
        filemode="a",
    ) """

    logger.info("Starting Script...")

    for file in os.listdir(source_dir):
        print(file)
        source_file = os.path.join(source_files, file)
        # output_file = os.path.join(output_subdir, file)

        file_extension = os.path.splitext(file)[1]
        print("file_extension", file_extension)

        if file_extension.lower() == ".pst":
            print(f"{source_file} Processing.")
            extract(output_dir, source_file)
            print(f"{source_file} Extracted.")
        else:
            print(f"The file {file}")



2023-11-21 18:43:35.885 | INFO     | __main__:<cell line: 1>:34 - Starting Script...
2023-11-21 18:43:35.888 | DEBUG    | __main__:extract:3 - Opening PST for processing...


3
Insufficient Parameters!
Matter Id: 108
Source Dir:  /content/drive/MyDrive/dataset
Output Dir:  /content/drive/MyDrive/dataset
TEST_DATA_FILE.pst
file_extension .pst
/content/drive/MyDrive/dataset/TEST_DATA_FILE.pst Processing.


2023-11-21 18:43:36.740 | DEBUG    | __main__:extract:7 - Starting traverse of PST structure...
2023-11-21 18:43:36.755 | DEBUG    | __main__:checkForMessages:2 - Processing Folder: SPAM Search Folder 2
2023-11-21 18:43:36.758 | WARNING  | __main__:folderReport:3 - Filename: /content/drive/MyDrive/dataset -->> Empty message not processed
2023-11-21 18:43:36.765 | DEBUG    | __main__:checkForMessages:2 - Processing Folder: Deleted Items
2023-11-21 18:43:36.768 | WARNING  | __main__:folderReport:3 - Filename: /content/drive/MyDrive/dataset -->> Empty message not processed
2023-11-21 18:43:36.772 | DEBUG    | __main__:checkForMessages:2 - Processing Folder: Inbox
2023-11-21 18:43:36.863 | DEBUG    | __main__:checkForMessages:2 - Processing Folder: Outbox
2023-11-21 18:43:36.866 | WARNING  | __main__:folderReport:3 - Filename: /content/drive/MyDrive/dataset -->> Empty message not processed
2023-11-21 18:43:36.868 | DEBUG    | __main__:checkForMessages:2 - Processing Folder: Sent Items
2023

/content/drive/MyDrive/dataset/TEST_DATA_FILE.pst Extracted.


In [ ]:
# with open('/content/sample_data/2023_11_10_05_16_49_Kawaljeet_Bhogal_0.txt') as f:
#     lines = f.read()